In [1]:
from IPython.core.display import HTML
import numpy as np
import matplotlib
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from pylab import *
import pysis
import os
import pickle

import scipy.optimize as sco
import time
import ast
from tkinter import *
from astropy.time import Time
%matplotlib qt
from IPython.core.display import HTML 
from PIL import Image
vims_wave = np.loadtxt('/home/alanyu/Dropbox (MIT)/VIMS_UROP/vims_wave.txt') 
HTML("<style>.container { width:96% !important; }</style>") 
from fitting_routine import *

/home/alanyu/anaconda3/lib/python3.8/site-packages/pysis/env.py:33: RuntimeWarning: Could not find isis. Is `ISISROOT` set?
  warnings.warn('Could not find isis. Is `ISISROOT` set?', RuntimeWarning)


In [2]:
import glob
import shapefile
from PIL import Image
iss = Image.open("Titan_ISS_P19658_Mosaic_Global_4km.tif")
plt.imshow(iss,extent=[0,360,-90,90], cmap='gray')

#sfiles = glob.glob("dune_shape_files/Belet.shp")
sfiles = glob.glob("dune_shape_files/*.shp")
print(sfiles)

sfiles = ['dune_shape_files/Shangrai-La.shp', 'dune_shape_files/Senkyo.shp', 'dune_shape_files/Belet.shp']
sfiles = ['dune_shape_files/Aztlan.shp', 'dune_shape_files/Fensal.shp']
for f in sfiles:
    sf = shapefile.Reader(f)
    for shape in sf.shapes():
        vertices = np.asarray(shape.points)
        ind = np.where(vertices[:,0]<0)
        vertices[ind,0] = vertices[ind,0]+360
        plt.scatter(vertices[:,0],vertices[:,1])
        
plt.show()

['dune_shape_files/Shangrai-La.shp', 'dune_shape_files/Aztlan.shp', 'dune_shape_files/Fensal.shp', 'dune_shape_files/Senkyo.shp', 'dune_shape_files/Belet.shp']


In [3]:
belet = glob.glob("pixels/belet/*.pkl")
senkyo = glob.glob("pixels/senkyo/*.pkl")
shangrai = glob.glob("pixels/shangrai/*.pkl")

In [4]:
#update()
def fit(info):
    """
    info: spectrum list.
    
    Returns: Parameters of fitting routine as a dictionary.
    """
    band_channels = [29,30,31,32,47,48,49,50,51,52,53,54,55,84,85,86,87,88,89,90] + [i for i in range(160,181)] 
    plaw = []
    #if info['latitude']<-90:
       # raise ValueError#, "NOT ON DISK"
#     else:
    try:
        p_fit,p_cov = sco.curve_fit(powerlaw,vims_wave[band_channels],info[band_channels],p0=(0.005,-2,0),maxfev=100000)
        p=powerlaw(vims_wave,*p_fit)
        #plaw.append(p_fit)
        #Subtract powerlaw
        adjusted = info-p

        #Fit Gaussian
        fit,cov=sco.curve_fit(gaussian,vims_wave[39:51],adjusted[39:51],p0=(1.6,.125,.1),maxfev=100000)
        #return fit[-1]
        return {'p_amp': p_fit[0],'p_power': p_fit[1],'p_const': p_fit[2],'g_mean':fit[0],'g_sigma': fit[1],'g_amp':fit[2]}
    except:
        #powerlaw fails, directly fit gaussian
        
        try:
            fit,cov = sco.curve_fit(gaussian, vims_wave[39:51],info[39:51],p0=(1.6,.125,.1),maxfev=100000)

    #return fit[-1]
    #plaw.append(None)

            return {'p_amp': np.nan,'p_power': np.nan,'p_const': np.nan,'g_mean':fit[0],'g_sigma': fit[1],'g_amp':fit[2]}# np.nan#fit[0],np.nan
        except: #no fit found
            return {}

In [5]:
def parse_csv(path_to_csv,channel=69):
    """
    path_to_csv: path to file
    channel: spectra of interest, default is 69.
    
    Returns dictionary containing desired data. Only works for * query.
    """
    count = 0
    data = []
    shape_name = path_to_csv.split("_")[0]
    bad = 0
    with open(path_to_csv,'r') as f:
        
        head = f.readline().replace('"','').strip().split(',')
        print('head',head)
        while True:
            
            try:
                entry = f.readline()
            except:
            #    print('2')
                break
                
            if entry == '':
                print('exit')
                break
                
            entry=entry.replace('"','').split('{')
            info = entry[0][:-1].split(',')
            #print(entry[0], '\n --- \n', entry[1])
            spectra = list(map(float,entry[1].split('}')[0].split(',')))
            #print(temp[0],'\n ---- \n', temp[1])
            #spectra = list(map(float,entry[1][:-2].split(',')))
            #print(spectra)
            entry_dict = dict()
            for i,col in enumerate(head[:-2]):
                if col not in ['name','sequenceid','observationid','starttime']:
                    entry_dict[col]=float(info[i])
                else:
                    entry_dict[col]=info[i]
            entry_dict['channel']=spectra[channel-1]
            entry_dict['spectrum'] = np.array(spectra)
            f_ = fit(entry_dict['spectrum'])
            if f_ == {}:
                bad+=1
                
                print('bad fit',bad,entry_dict['name'])
                #break
            entry_dict.update(f_)

            data.append(entry_dict)
            cube_name = entry_dict['name'].split('.cub')[0]
            latitude = entry_dict['latitude']
            longitude = entry_dict['longitude']
            file = open(f"pixels/{shape_name}/16_{shape_name}_{cube_name}_{latitude}_{longitude}.pkl","wb")
            pickle.dump(entry_dict,file)
            file.close()
            count+=1
            if count == 1:
                print(data)
                #break
            if count%1000 == 0:
                print("Progress",count)
            
            #print('DONE')
    print(count,bad)
    #return data
        

In [ ]:
parse_csv('senkyo_pixels.csv',45)

In [6]:
def import_pickle(*file_lists):
    micron_2 = []
    micron_12 = []
    micron_16 = []
    count = 0
    for file_list in file_lists:
        for i,file in enumerate(file_list):
            
            wind = file.split('/')[2][:2]
            with open(file,"rb") as f:
                if wind == '12':
                    
                    micron_12.append(pickle.load(f))
                elif wind == '16':
                  
                    micron_16.append(pickle.load(f))
                else:
                    micron_2.append(pickle.load(f))
            count+=1
            if count%10000==0: print(count,len(micron_12),len(micron_16),len(micron_2))
    return micron_2, micron_12, micron_16
micron_2, micron_12, micron_16 = import_pickle(belet,senkyo,shangrai)

10000 3368 3291 3341
20000 6705 6554 6741
30000 10059 9790 10151
40000 13352 13126 13522
50000 16595 16510 16895
60000 19863 19889 20248
70000 23187 23274 23539
80000 26512 26704 26784
90000 29857 29966 30177
100000 33287 33263 33450
110000 36595 36590 36815
120000 39984 39952 40064
130000 43308 43319 43373
140000 46673 46696 46631
150000 50005 49999 49996
160000 53365 53320 53315
170000 56755 56648 56597
180000 60115 59974 59911
190000 63436 63264 63300
200000 66749 66582 66669
210000 70154 69862 69984
220000 73477 73220 73303
230000 76846 76512 76642
240000 80214 79809 79977
250000 83567 83098 83335
260000 86851 86447 86702
270000 90104 89842 90054
280000 93423 93159 93418
290000 96692 96456 96852
300000 100006 99815 100179
310000 103346 103129 103525
320000 106705 106426 106869
330000 110050 109760 110190
340000 113353 113136 113511
350000 116692 116466 116842
360000 120014 119820 120166
370000 123365 123117 123518
380000 126685 126480 126835
390000 129995 129820 130185
400000 13330

In [8]:
micron_2[0]['spectrum']

array([ 0.0763505 ,  0.113136  ,  0.138922  ,  0.146524  ,  0.126     ,
        0.0936954 ,  0.0795553 ,  0.0763359 ,  0.0885651 ,  0.111927  ,
        0.131274  ,  0.140332  ,  0.13276   ,  0.100027  ,  0.0593369 ,
        0.0431969 ,  0.0295527 ,  0.0313845 ,  0.042422  ,  0.0622391 ,
        0.0827431 ,  0.0844537 ,  0.0981416 ,  0.109633  ,  0.124524  ,
        0.0986692 ,  0.0649221 ,  0.046782  ,  0.0333819 ,  0.0279256 ,
        0.0251519 ,  0.0251015 ,  0.0311578 ,  0.0397409 ,  0.046074  ,
        0.0519955 ,  0.0599261 ,  0.0705508 ,  0.0756816 ,  0.0824274 ,
        0.0849288 ,  0.0843681 ,  0.0910672 ,  0.0900338 ,  0.0759746 ,
        0.0561447 ,  0.0431159 ,  0.029171  ,  0.0267057 ,  0.0200695 ,
        0.0187427 ,  0.0184399 ,  0.0240378 ,  0.0220268 ,  0.0189923 ,
        0.0214212 ,  0.0293037 ,  0.0326464 ,  0.0306571 ,  0.036423  ,
        0.0447716 ,  0.0463914 ,  0.0401177 ,  0.0389254 ,  0.0412627 ,
        0.0458172 ,  0.0524996 ,  0.0612893 ,  0.067655  ,  0.06

In [43]:
def get_list_data(pixels,sample = 50,cuts=False):
    """
    Given list of pixels, returns list representations to be used for scatter plot.
    """
    bad_list = {'V1585148900_2.ir_rc20final.cub', 'V1563500954_1_001.ir_rc20final.cub', 'V1563500646_1_001.ir_rc20final.cub', 'V1585148917_2.ir_rc20final.cub', 'V1640658864_4.ir_rc20final.cub', 'V1585148968_2.ir_rc20final.cub', 'V1585148955_2.ir_rc20final.cub', 'V1585148951_2.ir_rc20final.cub', 'V1640659304_4.ir_rc20final.cub', 'V1585148904_2.ir_rc20final.cub', 'V1585148934_2.ir_rc20final.cub', 'V1548750098_1.ir_rc20final.cub', 'V1563500731_1_001.ir_rc20final.cub', 'V1585148947_2.ir_rc20final.cub', 'V1585148942_2.ir_rc20final.cub', 'V1563500693_1_001.ir_rc20final.cub', 'V1563500714_1_002.ir_rc20final.cub', 'V1563500744_1_001.ir_rc20final.cub', 'V1585148930_2.ir_rc20final.cub', 'V1563500620_1_002.ir_rc20final.cub'}
    bad_list2 = {'V1563500984_1_002.ir_rc20final.cub', 'V1563500826_1_002.ir_rc20final.cub', 'V1563500821_1_001.ir_rc20final.cub', 'V1563500633_1_001.ir_rc20final.cub', 'V1563500624_1_002.ir_rc20final.cub', 'V1563500804_1_001.ir_rc20final.cub', 'V1563500804_1_002.ir_rc20final.cub', 'V1563500658_1_001.ir_rc20final.cub', 'V1563500663_1_002.ir_rc20final.cub', 'V1563500667_1_002.ir_rc20final.cub', 'V1563501002_1_001.ir_rc20final.cub', 'V1563500783_1_002.ir_rc20final.cub', 'V1574150707_3.ir_rc20final.cub', 'V1563500847_1_001.ir_rc20final.cub'}
    bad_list |= bad_list2
    bad_list3 = {'V1563500594_1_001.ir_rc20final.cub', 'V1563500972_1_002.ir_rc20final.cub', 'V1563500873_1_001.ir_rc20final.cub', 'V1563500680_1_002.ir_rc20final.cub', 'V1563500997_1_001.ir_rc20final.cub', 'V1563500654_1_003.ir_rc20final.cub', 'V1563500688_1_003.ir_rc20final.cub', 'V1563500714_1_001.ir_rc20final.cub', 'V1649176775_1.ir_rc20final.cub', 'V1563500693_1_002.ir_rc20final.cub', 'V1563500757_1_003.ir_rc20final.cub', 'V1563500779_1_003.ir_rc20final.cub', 'V1563500834_1_001.ir_rc20final.cub'}
    bad_list |= bad_list3
    bad_list4 = {'V1563500744_1_003.ir_rc20final.cub', 'V1563500774_1_001.ir_rc20final.cub', 'V1563500654_1_001.ir_rc20final.cub', 'V1563500663_1_001.ir_rc20final.cub', 'V1563500688_1_002.ir_rc20final.cub'}
    bad_list |= bad_list4
    time=[]
    spectra=[]
    phase=[]
    incidence=[]
    emission=[]
    latitude=[]
    longitude=[]
    g_amp=[]
    g_mean=[]
    g_sigma=[]
    p_amp=[]
    p_power=[]
    p_const=list()
    cut=0
    bad_cubes = set()
    num_bad = 0
    for i,entry in enumerate(pixels):
        if i%sample==0:
            if entry['name'] not in bad_list and (not cuts or (cuts and (20<entry['incidence'] < 30 and 20<entry['emission'] < 30))):
                if entry.get('g_amp') == None:
                    continue
                
                spectra.append(entry['channel'])
                g_amp.append(entry['g_amp'])
                g_mean.append(entry['g_mean'])
                g_sigma.append(entry['g_sigma'])
                p_amp.append(entry['p_amp'])
                p_power.append(entry['p_power'])
                p_const.append(entry['p_const'])

                phase.append(entry['phase'])
                incidence.append(entry['incidence'])
                emission.append(entry['emission'])
                latitude.append(entry['latitude'])
                longitude.append(entry['longitude'])
                time.append(entry['ettime'])
                
                if g_mean[-1] < 2 or g_mean[-1] > 2.1 or g_sigma[-1] < 0 or g_sigma[-1] < .03:
                    bad_cubes.add(entry['name'])
                    num_bad += 1
                
            elif cuts:
                cut+=1
    print('Cut from sample', cut, 'of',len(pixels)//sample)
    print(bad_cubes,num_bad)    
    return {'time':time,'spectra':spectra,'g_amp':g_amp,'g_mean':g_mean,'g_sigma':g_sigma,'p_amp':p_amp,'p_power':p_power,'p_const':p_const,\
           'phase':phase,'incidence':incidence,'emission':emission,'latitude':latitude,'longitude':longitude}
    

In [44]:
len(micron_12)

279792

In [45]:
def plot_two_micron(pixels,gaussian_amp=False,sample=50,cuts=False):
    L = get_list_data(pixels,sample,cuts)
    bad_pixels = 0
    if gaussian_amp:
        spec = L['g_amp']
    else:
        spec = L['spectra']
    time = L['time']
    incidence,emission,phase = L['incidence'],L['emission'],L['phase']
    amps,t,inc,emi,pha = [],[],[],[],[]
    for i,e in enumerate(spec):
        if e < 0 :
            bad_pixels+=1
            continue
        
        #amps.append(e*np.cos(incidence[i]*np.pi/180)/(np.cos(incidence[i]*np.pi/180)+np.cos(emission[i]*np.pi/180)))
        amps.append(e)
        
        t.append(time[i])
        inc.append(incidence[i])
        emi.append(emission[i])
        pha.append(phase[i])
    
    print(len(t),len(amps), f"Bad pixels: {bad_pixels}")
    
    plt.scatter(t,amps,c=pha,alpha=0.3)
    plt.colorbar()
    plt.xticks(rotation="45", fontsize='8')
    plt.xlabel("Time")
    plt.ylabel("Channel")
    plt.ylim([0,0.1])
    plt.title("Spectra vs. Time")
    return amps, t

In [50]:
plot_two_micron(micron_12,True,cuts=True)
#1.2 micron window has a lot of bad gaussian fits

Cut from sample 5447 of 5595
{'V1639297035_1.ir_rc20final.cub', 'V1717756176_1.ir_rc20final.cub', 'V1716369091_1.ir_rc20final.cub', 'V1561844852_1.ir_rc20final.cub', 'V1559083653_1.ir_rc20final.cub', 'V1640670042_1.ir_rc20final.cub', 'V1575515382_1.ir_rc20final.cub', 'V1557715091_1.ir_rc20final.cub', 'V1643436171_1.ir_rc20final.cub', 'V1557713993_1.ir_rc20final.cub', 'V1575514957_1.ir_rc20final.cub', 'V1575510641_1.ir_rc20final.cub', 'V1640663699_1.ir_rc20final.cub', 'V1640658214_4.ir_rc20final.cub', 'V1657178597_1.ir_rc20final.cub', 'V1561844726_1.ir_rc20final.cub', 'V1664054617_1.ir_rc20final.cub', 'V1657179290_1.ir_rc20final.cub', 'V1556340329_1.ir_rc20final.cub', 'V1561841124_3.ir_rc20final.cub', 'V1557712109_1.ir_rc20final.cub', 'V1702522357_1.ir_rc20final.cub', 'V1557722963_1.ir_rc20final.cub', 'V1561840514_1.ir_rc20final.cub', 'V1664053498_1_002.ir_rc20final.cub', 'V1702505572_1.ir_rc20final.cub', 'V1559083936_1.ir_rc20final.cub', 'V1716366697_1.ir_rc20final.cub', 'V1721869489_1

([0.07838846168355552,
  0.07068085630583387,
  0.07260894071873641,
  0.07116952147568707,
  0.07499840069605365,
  0.07392318476795448,
  0.08031273331990553,
  0.06850746911554176,
  0.07602930925294024,
  0.07416135823200116,
  0.07396618109453812,
  0.06698644916061477,
  0.07266026786346176,
  0.07361484873183033,
  0.06823957277833834,
  0.0987762826761699,
  0.07388561538366047,
  0.09364731274155999,
  0.08846967065770187,
  0.07326836252019785,
  0.08246000877403833,
  0.08133413801623532,
  0.08234756808313964,
  0.07393925101791181,
  0.08518266761092826,
  0.07654261636772172,
  0.07031203138732689,
  0.07143732476359804,
  0.07392730845607884,
  0.08546541527013472,
  7.182959954290746e-18,
  0.08799033132809018,
  0.08720318946954599,
  0.07309059665625524,
  0.0739329844569096,
  0.0799803575958743,
  0.08898896522131199,
  0.08456408867491078,
  0.0721277989960907,
  0.0797003519889366,
  0.07728009975297309,
  0.07367435107937309,
  0.0687556943796505,
  0.09126588103

In [55]:
def plot_windows(window1,window2,cuts=False):
    L1 = get_list_data(window1,1,cuts)['g_mean']
    L2 = get_list_data(window2,1,cuts)['g_mean']
    print(len(L1),len(L2))
    
    plt.scatter(L1,L2[:len(L1)],alpha = 0.3)
    plt.xlim([1.1,1.3])
    plt.ylim([1.5,1.7])

In [56]:
plot_windows(micron_12,micron_16,cuts=True)

Cut from sample 271493 of 279792
{'V1561844663_1.ir_rc20final.cub', 'V1664076421_1.ir_rc20final.cub', 'V1639297035_1.ir_rc20final.cub', 'V1683615420_1.ir_rc20final.cub', 'V1721869612_1.ir_rc20final.cub', 'V1559083653_1.ir_rc20final.cub', 'V1702516410_1.ir_rc20final.cub', 'V1643437619_1.ir_rc20final.cub', 'V1557715091_1.ir_rc20final.cub', 'V1561844537_1.ir_rc20final.cub', 'V1557713605_1.ir_rc20final.cub', 'V1554969990_1.ir_rc20final.cub', 'V1639284162_1.ir_rc20final.cub', 'V1575514957_1.ir_rc20final.cub', 'V1655788743_1.ir_rc20final.cub', 'V1559086977_1.ir_rc20final.cub', 'V1640658214_4.ir_rc20final.cub', 'V1561844726_1.ir_rc20final.cub', 'V1559087469_1.ir_rc20final.cub', 'V1664075372_1.ir_rc20final.cub', 'V1643444524_1.ir_rc20final.cub', 'V1643438092_1.ir_rc20final.cub', 'V1561840514_1.ir_rc20final.cub', 'V1664053498_1_002.ir_rc20final.cub', 'V1556340873_1.ir_rc20final.cub', 'V1639306778_1.ir_rc20final.cub', 'V1640686806_1.ir_rc20final.cub', 'V1556347600_1.ir_rc20final.cub', 'V15577224

Cut from sample 271493 of 279792
{'V1561844663_1.ir_rc20final.cub', 'V1664076421_1.ir_rc20final.cub', 'V1639297035_1.ir_rc20final.cub', 'V1683615420_1.ir_rc20final.cub', 'V1721869612_1.ir_rc20final.cub', 'V1559083653_1.ir_rc20final.cub', 'V1702516410_1.ir_rc20final.cub', 'V1643437619_1.ir_rc20final.cub', 'V1557715091_1.ir_rc20final.cub', 'V1561844537_1.ir_rc20final.cub', 'V1557713605_1.ir_rc20final.cub', 'V1554969990_1.ir_rc20final.cub', 'V1639284162_1.ir_rc20final.cub', 'V1575514957_1.ir_rc20final.cub', 'V1655788743_1.ir_rc20final.cub', 'V1559086977_1.ir_rc20final.cub', 'V1640658214_4.ir_rc20final.cub', 'V1559087469_1.ir_rc20final.cub', 'V1561844726_1.ir_rc20final.cub', 'V1664075372_1.ir_rc20final.cub', 'V1643444524_1.ir_rc20final.cub', 'V1643438092_1.ir_rc20final.cub', 'V1561840514_1.ir_rc20final.cub', 'V1664053498_1_002.ir_rc20final.cub', 'V1556340873_1.ir_rc20final.cub', 'V1640686806_1.ir_rc20final.cub', 'V1556347600_1.ir_rc20final.cub', 'V1639306778_1.ir_rc20final.cub', 'V15577224

In [57]:
def plot_temporal(pixels,sample=50,cuts=False):
    """
    Given list of pixels, returns temporal plots of best fit parameters (6). 
    """
    fig, axs = plt.subplots(2, 3)
    
    L = get_list_data(pixels,sample,cuts)
    bad_pixels = 0
    g_amp,g_mean,g_sigma,p_amp,p_power,p_const = L['g_amp'],L['g_mean'],L['g_sigma'],L['p_amp'],L['p_power'],L['p_const']
    time = L['time']
    incidence,emission,phase = L['incidence'],L['emission'],L['phase']
    
    gamp,gmean,gsigma,pamp,ppower,pconst = [],[],[],[],[],[]
    t,inc,emi,pha = [],[],[],[]
    for i,e in enumerate(g_amp):
        if e < 0: #or e>1.5:
            bad_pixels+=1
            continue
        
        #gamp.append(e*np.cos(incidence[i]*np.pi/180)/(np.cos(incidence[i]*np.pi/180)+np.cos(emission[i]*np.pi/180)))
        gamp.append(e)
        gmean.append(g_mean[i])
        gsigma.append(g_sigma[i])
        pamp.append(p_amp[i])
        ppower.append(p_power[i])
        pconst.append(p_const[i])
        
        t.append(time[i])
        inc.append(incidence[i])
        emi.append(emission[i])
        pha.append(phase[i])
    
    axs[0, 0].scatter(t, gamp,alpha=0.3)
    axs[0, 0].set_title('Gaussian Amplitude')
    axs[0, 0].set_ylim([0,0.2])
    
    axs[0, 1].scatter(t, gmean, alpha=0.3)
    axs[0, 1].set_title('Gaussian Mean')
    axs[0, 1].set_ylim([1.4,1.6])
    p = np.polyfit(t,gmean, 1)
    fit = np.polyval(p, t)
    axs[0, 1].plot(t,fit, c='r')
    print(fit[0],fit[-1])
    
    axs[0, 2].scatter(t, gsigma,alpha=0.3)
    axs[0, 2].set_title('Gaussian Sigma')
    axs[0, 2].set_ylim([-0.05,.25])
    
    axs[1, 0].scatter(t, pamp,alpha=0.3)
    axs[1, 0].set_title('Powerlaw Amplitude')
    axs[1, 0].set_ylim([-.1,.25])

    axs[1, 1].scatter(t, ppower,alpha=0.3)
    axs[1, 1].set_title('Powerlaw Exponent')
    axs[1, 1].set_ylim([-15,10])

    axs[1, 2].scatter(t, pconst,alpha=0.3)
    axs[1, 2].set_title('Powerlaw Constant')
    axs[1, 2].set_ylim([-.02,.02])

#     axs[2, 0].scatter(gamp,gsigma,alpha = 0.3)
#     axs[2, 0].set_title('Gamp vs Gsigma')
#     axs[2, 0].set_ylim([0,0.4])
#     axs[2, 0].set_xlim([0,0.3])
    
#     axs[2, 1].scatter(gmean,gamp,alpha=0.3)
#     axs[2, 1].set_title('Gmean vs Gamp')
#     axs[2,1].set_ylim([0,.3])
#     axs[2, 2].scatter(gmean,gsigma,alpha=0.3)
#     axs[2, 2].set_title('Gmean vs Gsigma')
#     p = np.polyfit(gmean,gsigma,1)
#     fit = np.polyval(p,gmean)
#     axs[2,2].plot(gmean,fit,c='b')
#     axs[2,2].set_ylim([0,0.2])
    
#     axs[3,0].scatter(pha, gamp,alpha=0.3)
#     axs[3,0].set_ylim([0,0.2])
    return bad_pixels/len(gamp)
    

In [4]:
#1.2,1.6,2.0,1.0,9.38

#def fit powerlaw to channel < 90, fit gaussian to all five windows, throw out entire spectra if parameters are bad

#experiment with viewing geometry cuts, lower might be better, limit range to ~ 20 deg but make sure enough data

#average spectrum over time buckets (mean)
#1) average fits
#2) average spectra and then fit
#3) average of all data
#one hump for six month period

#rainbow effect


ncolors = 20 
colors = plt.cm.nipy_spectral(np.linspace(0.05,0.95,ncolors)) 

for i in range(20):
    mean = i*.1
    sigma = .2
    x = np.linspace(-5,5,1000)
    G = gaussian(x,mean,sigma,1)
    plt.plot(x,G,color=colors[i])
    

In [2]:
from scipy.odr import Model,Data,ODR
linear = Model(lambda B,x: B[0]*x+B[1]) 

def linear_ols_fit(x,y,linearity_cut=np.inf,cut_x=False): 
    '''
    Ordinary least squares linear fitting routine.  Typically used for fitting
    the Signal vs Variance plot for the Photon Transfer.
    
    N.B. The linearity cut is defaulted to the x variable, as this is the
    funtion used for the Signal vs Variance.
    
    Inputs:
        
        x - independent variable to fit
        
        y - dependent variable 
        
        linearity_cut - value at which to cut the linear fit. Default True.
        
        cut_x - apply the linearity cut to the independent variable.  If False,
        the cut is applied to the dependent variable.
        
    Outputs:
        
        m - slope of the best fit line
        
        m_err - error on the best fit slope
        
        b - offset of the best fit line
        
        b_err - error on the best fit offset
    '''   
    
    #First determine which indices meets the linearity cut
    if cut_x:
        linear_ind = np.where(x<linearity_cut)[0]
    else:
        linear_ind = np.where(y<linearity_cut)[0]
    
    #Provide the data to the scipy.odr solver    
    data = Data(x[linear_ind],y[linear_ind])
    #Set up the ODR.  Intial guesses are provided from the first two entried
    #of the supplied variables.  The fit is not very sensitive to initial guess.
    odr = ODR(data,linear,beta0 = [(y[1]-y[0])/(x[1]-x[0]),y[0]])
    #Run the solver
    result = odr.run()
    #Grab the result
    m,b = result.beta
    #Grab the errors in the parameters
    m_err,b_err = result.sd_beta
    return(m,m_err,b,b_err)     


NameError: name 'np' is not defined

In [58]:
plot_temporal(micron_16,1,True)

Cut from sample 271493 of 279792
{'V1561844663_1.ir_rc20final.cub', 'V1664076421_1.ir_rc20final.cub', 'V1639297035_1.ir_rc20final.cub', 'V1683615420_1.ir_rc20final.cub', 'V1721869612_1.ir_rc20final.cub', 'V1559083653_1.ir_rc20final.cub', 'V1702516410_1.ir_rc20final.cub', 'V1643437619_1.ir_rc20final.cub', 'V1557715091_1.ir_rc20final.cub', 'V1561844537_1.ir_rc20final.cub', 'V1557713605_1.ir_rc20final.cub', 'V1554969990_1.ir_rc20final.cub', 'V1639284162_1.ir_rc20final.cub', 'V1575514957_1.ir_rc20final.cub', 'V1655788743_1.ir_rc20final.cub', 'V1559086977_1.ir_rc20final.cub', 'V1640658214_4.ir_rc20final.cub', 'V1559087469_1.ir_rc20final.cub', 'V1561844726_1.ir_rc20final.cub', 'V1664075372_1.ir_rc20final.cub', 'V1643444524_1.ir_rc20final.cub', 'V1643438092_1.ir_rc20final.cub', 'V1561840514_1.ir_rc20final.cub', 'V1664053498_1_002.ir_rc20final.cub', 'V1556340873_1.ir_rc20final.cub', 'V1640686806_1.ir_rc20final.cub', 'V1556347600_1.ir_rc20final.cub', 'V1639306778_1.ir_rc20final.cub', 'V15577224

1.5685720123420046 1.5711653622436248


0.0

In [1]:
import glob
import shapefile
from PIL import Image
iss = Image.open("Titan_ISS_P19658_Mosaic_Global_4km.tif")
plt.imshow(iss,extent=[0,360,-90,90], cmap='gray')

#sfiles = glob.glob("dune_shape_files/Belet.shp")
sfiles = glob.glob("dune_shape_files/*.shp")
print(sfiles)
for f in sfiles:
    sf = shapefile.Reader(f)
    for shape in sf.shapes():
        vertices = np.asarray(shape.points)
        ind = np.where(vertices[:,0]<0)
        vertices[ind,0] = vertices[ind,0]+360
        plt.scatter(vertices[:,0],vertices[:,1])
        
for i,pixel in enumerate(pixels):
    if i%500==0:
        plt.scatter(pixel['longitude'],pixel['latitude'])
plt.show()

NameError: name 'plt' is not defined